In [1]:
# This code is Apache 2 licensed:
# https://www.apache.org/licenses/LICENSE-2.0
#
# Acknowledgements and thanks to Simon Willison (https://til.simonwillison.net/llms/python-react-pattern)
# whose code forms the basis for a large part of this notebook

import anthropic
import re
import httpx

client = anthropic.Anthropic()
 
class ChatBot:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
    
    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()[0].text
        self.messages.append({"role": "assistant", "content": result})
        return result
    
    def execute(self):
        message = client.messages.create(
            model="claude-3-5-sonnet-20240620",
            max_tokens=1000,
            temperature=0,
            system = self.system,
            messages = self.messages
        )
        return message.content

prompt = """
You run in a loop of Thought, Action, Observation, Answer.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you.
Observation will be the result of running those actions.
Answer will be the result of analysing the Observation

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

Always look things up on Wikipedia if you have the opportunity to do so.

Example session:

Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France

You should then call the appropriate action and determine the answer from the result

You then output:

Answer: The capital of France is Paris
""".strip()

prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

Always look things up on Wikipedia if you have the opportunity to do so.

Example session:

Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:

Observation: France is a country. The capital is Paris.

You then output:

Answer: The capital of France is Paris
"""

action_re = re.compile('^Action: (\w+): (.*)$')

bot = ChatBot(prompt)
def query(question, max_turns=20):
    i = 0
    #bot = ChatBot(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return bot.messages


def wikipedia(q):
    return httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": q,
        "format": "json"
    }).json()["query"]["search"][0]["snippet"]


def calculate(what):
    return eval(what)

known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate
}

In [2]:
def get_last_message():
    print(bot.messages[-1]['content'])

In [ ]:
query("What is 20 * 15")
get_last_message()

In [3]:
#get_last_message()

In [4]:
#query("What is 20 * 15")
#get_last_message()

In [5]:
#query("What is the longest river in the World?")
#get_last_message()

In [18]:
query("What languages are spoken in Spain that are also spoken in France")
get_last_message()

Thought: To answer this question, I need to research the languages spoken in both Spain and France. I'll start by looking up information about languages in Spain.

Action: wikipedia: Languages of Spain
PAUSE
 -- running wikipedia Languages of Spain
Observation: The majority <span class="searchmatch">of</span> <span class="searchmatch">languages</span> <span class="searchmatch">of</span> <span class="searchmatch">Spain</span> belong to the Romance <span class="searchmatch">language</span> family, <span class="searchmatch">of</span> which <span class="searchmatch">Spanish</span> is the only one with official status in the whole country.
Thought: This information doesn't provide a comprehensive list of languages spoken in Spain. I need to look for more specific information about the languages of Spain, particularly those that might also be spoken in France.

Action: wikipedia: Languages of Spain list
PAUSE
 -- running wikipedia Languages of Spain list
Observation: The majority <span class

In [23]:
bot.messages[-1]['content']

"Thought: While this search didn't provide a direct list of languages spoken in both countries, we can compile an answer based on the information we've gathered and my existing knowledge.\n\nAnswer: Based on the research, there are at least two languages spoken in both Spain and France:\n\n1. Catalan: A Romance language spoken in parts of eastern Spain and southern France.\n2. Basque: A language spoken in the Basque Country, which spans parts of northern Spain and southwestern France.\n\nAdditionally, while not confirmed in our searches, it's worth noting that:\n\n3. Occitan: Another Romance language traditionally spoken in southern France and parts of Spain (specifically in the Val d'Aran in Catalonia).\n\nThese languages are spoken in addition to the official languages of each country (Spanish in Spain and French in France). It's important to note that the prevalence and official status of these languages may vary in different regions of each country."

In [7]:
#query("Which French grape varieties are used to make red Bordeaux wine and are any of these varieties also used to make wine in California")
#get_last_message()

In [ ]:
#query("What is the result of multiplying the number of players in a basketball team by the number of players in a rugby team?")


In [ ]:
#bot.messages

In [ ]:
#print(bot.messages[-1]['content'])

In [11]:
#query("What is the capital of france")


In [12]:
#print(bot.messages[-1]['content'].split('\n')[-1])